In [23]:
import pickle
import pandas as pd

In [53]:
import pickle


archivo_pickle = 'base_epsilon_1.0.pkl'


with open(archivo_pickle, 'rb') as file:
    df = pickle.load(file)


In [54]:
df['Enfermedad'] = df['Enfermedad'].str.split(',')

# Crear variables dummies
enfermedades_dummies = df['Enfermedad'].str.join('|').str.get_dummies()

In [55]:
df.tail()

,EPS,SEXO,DPTO,MUNI,DIAS_COMP,V8,Valor_Neto,DIAS_ESTAN_Neto,Edad,Grupo_Edad,Zona,Enfermedad
rownames,,,,,,,,,,,,
14721902,EPS002,F,76,776,360,B,96086.646585,37.645191,31.374860,3,N,[ENFERMEDADCARDIOVASCULAR_OTRA]
9275642,EPS013,F,50,1,215,C,2.080607,0.300281,49.351731,3,N,[ENFERMEDADCARDIOVASCULAR_OTRA]
600067,EPS002,M,50,776,30,A,-0.068496,1.577482,29.897231,3,N,[ANOMALIASGENETICASYCONGENITAS]
1094959,EPS002,F,50,776,360,B,70112.537603,-0.535148,32.285434,3,N,[Sin Enfermedad]
1486393,EPS017,M,50,776,30,B,1.315701,-1.081141,46.183017,3,N,[ENFERMEDADCARDIOVASCULAR_OTRA]


In [56]:


# Fusionar las dummies con el DataFrame original
df = pd.concat([df.drop('Enfermedad', axis=1), enfermedades_dummies], axis=1)

# Asegurarse de que "Sin Enfermedad" deje todas las demás columnas de enfermedad en 0
if 'Sin Enfermedad' in df.columns:
    for col in enfermedades_dummies.columns:
        if col != 'Sin Enfermedad':
            df.loc[df['Sin Enfermedad'] == 1, col] = 0

# Eliminar la columna "Sin Enfermedad" si no la necesitas más
df = df.drop('Sin Enfermedad', axis=1)

In [57]:
df.head()

,EPS,SEXO,DPTO,MUNI,DIAS_COMP,V8,Valor_Neto,DIAS_ESTAN_Neto,Edad,Grupo_Edad,Zona,ANOMALIASGENETICASYCONGENITAS,ARTROSIS,ASMA,CANCER_CERVIXINSITU,CANCER_GENITALESMASCULINOS,CANCER_MAMA,CANCER_MELANOMAYDELAPIEL,DIABETES,ENFERMEDADCARDIOVASCULAR_HIPERTENSION,ENFERMEDADCARDIOVASCULAR_OTRA
rownames,,,,,,,,,,,,,,,,,,,,,
1184873,EPS017,M,50,776,360,C,39052.524650,-3.873718,10.336085,5,N,0,0,0,0,0,0,0,0,0,1
1587467,EPS002,F,50,317,336,B,312012.713584,276.944493,22.477167,3,N,0,0,0,0,0,0,0,0,0,0
753062,EPS013,M,50,776,333,C,14312.511115,1.165817,13.361120,3,N,0,0,0,0,0,0,0,0,0,1
1802563,EPS013,F,50,776,84,B,0.754629,0.182802,12.688437,5,N,1,0,0,0,0,0,0,0,0,0
1190692,EPS002,F,50,1,180,C,-1.241827,-0.085316,20.357126,3,N,0,0,0,0,0,0,0,0,0,1


In [39]:
df.columns

Index(['EPS', 'SEXO', 'DPTO', 'MUNI', 'DIAS_COMP', 'V8', 'Valor_Neto',
       'DIAS_ESTAN_Neto', 'Edad', 'Grupo_Edad', 'Zona',
       'ANOMALIASGENETICASYCONGENITAS', 'ARTROSIS', 'ASMA',
       'CANCER_CERVIXINSITU', 'CANCER_GENITALESMASCULINOS', 'CANCER_MAMA',
       'CANCER_MELANOMAYDELAPIEL', 'DIABETES',
       'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
       'ENFERMEDADCARDIOVASCULAR_OTRA'],
      dtype='object')

In [58]:
pd.set_option('display.max_columns', None)
df.shape

(996234, 21)

In [59]:
# Eliminar columnas donde todas las entradas son cero
df = df.loc[:, (df != 0).any(axis=0)]
df['DIAS_COMP'] = df['DIAS_COMP'].astype(float)
df.drop(columns='MUNI', inplace=True)

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


df = df.dropna()

# Binarizar las variables binarias especificadas


# Aplicar One-Hot Encoding a las variables categóricas
categorical_columns = ['EPS', 'SEXO', 'DPTO', 'V8', 'Grupo_Edad', 'Zona']
encoder = OneHotEncoder()
encoded_categoricals = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_categoricals.toarray(), columns=encoder.get_feature_names_out(categorical_columns), index=df.index)

# Concatenar los datos codificados con el resto del dataframe
df = pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1)

# Dividir el conjunto de datos en entrenamiento y prueba
X = df.drop('Valor_Neto', axis=1)
y = df['Valor_Neto']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

# Ahora X_train, X_test, y_train, y_test están listos para ser usados en el modelado.


In [61]:
X_train.head()

,DIAS_COMP,DIAS_ESTAN_Neto,Edad,ANOMALIASGENETICASYCONGENITAS,ARTROSIS,ASMA,CANCER_CERVIXINSITU,CANCER_GENITALESMASCULINOS,CANCER_MAMA,CANCER_MELANOMAYDELAPIEL,DIABETES,ENFERMEDADCARDIOVASCULAR_HIPERTENSION,ENFERMEDADCARDIOVASCULAR_OTRA,EPS_EPS002,EPS_EPS008,EPS_EPS009,EPS_EPS013,EPS_EPS014,EPS_EPS016,EPS_EPS017,EPS_EPS018,EPS_EPS033,EPS_EPS037,SEXO_F,SEXO_M,DPTO_11,DPTO_13,DPTO_15,DPTO_25,DPTO_44,DPTO_50,DPTO_54,DPTO_76,DPTO_8,V8_A,V8_B,V8_C,Grupo_Edad_3,Grupo_Edad_5,Zona_N
rownames,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23686,330.0,13.848908,1.041305,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1367515,360.0,27.015069,10.164111,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
1370562,360.0,-1.105384,93.386420,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1329074,360.0,27.992105,25.263128,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
9978211,150.0,1.541955,16.202274,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [62]:

pd.set_option('display.max_rows', None)
X_train.dtypes

DIAS_COMP                                float64
DIAS_ESTAN_Neto                          float64
Edad                                     float64
ANOMALIASGENETICASYCONGENITAS              int64
ARTROSIS                                   int64
ASMA                                       int64
CANCER_CERVIXINSITU                        int64
CANCER_GENITALESMASCULINOS                 int64
CANCER_MAMA                                int64
CANCER_MELANOMAYDELAPIEL                   int64
DIABETES                                   int64
ENFERMEDADCARDIOVASCULAR_HIPERTENSION      int64
ENFERMEDADCARDIOVASCULAR_OTRA              int64
EPS_EPS002                               float64
EPS_EPS008                               float64
EPS_EPS009                               float64
EPS_EPS013                               float64
EPS_EPS014                               float64
EPS_EPS016                               float64
EPS_EPS017                               float64
EPS_EPS018          

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Inicializar los modelos
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(verbosity=0)
}

# Entrenar y evaluar los modelos
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    results[name] = {
        'train': {
            'R2': r2_score(y_train, y_train_pred),
            'MAE': mean_absolute_error(y_train, y_train_pred),
            'MSE': mean_squared_error(y_train, y_train_pred)
        },
        'test': {
            'R2': r2_score(y_test, y_test_pred),
            'MAE': mean_absolute_error(y_test, y_test_pred),
            'MSE': mean_squared_error(y_test, y_test_pred)
        }
    }

# Imprimir resultados
for model_name, metrics in results.items():
    print(f"Modelo: {model_name}")
    for dataset_type, values in metrics.items():
        print(f"\t{dataset_type} Metrics:")
        for metric_name, metric_value in values.items():
            print(f"\t\t{metric_name}: {metric_value:.4f}")

Modelo: Linear Regression
	train Metrics:
		R2: 0.0019
		MAE: 188371.7436
		MSE: 30131943829703.1172
	test Metrics:
		R2: 0.0368
		MAE: 182974.4554
		MSE: 406491199650.0234
Modelo: Decision Tree
	train Metrics:
		R2: 1.0000
		MAE: 0.0000
		MSE: 0.0000
	test Metrics:
		R2: -0.3941
		MAE: 178777.6678
		MSE: 588299488617.7722
Modelo: Random Forest
	train Metrics:
		R2: 0.8325
		MAE: 55421.6653
		MSE: 5058272945844.4902
	test Metrics:
		R2: -2.9222
		MAE: 141199.5420
		MSE: 1655199635359.5369
Modelo: XGBoost
	train Metrics:
		R2: 0.9937
		MAE: 122609.7005
		MSE: 190118987287.6265
	test Metrics:
		R2: -0.3254
		MAE: 131515.7954
		MSE: 559313658995.2170


In [65]:
!pip install optuna

In [66]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'tree_method': 'exact',  # para modelos más pequeños, 'exact' es adecuado
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    model = XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=50)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

best_params = study.best_params
print("Mejores parámetros:", best_params)

# Entrenar el modelo con los mejores parámetros
optimized_model = XGBRegressor(**best_params)
optimized_model.fit(X_train, y_train)

# Evaluación del modelo
y_train_pred = optimized_model.predict(X_train)
y_test_pred = optimized_model.predict(X_test)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"R2 Train: {train_r2:.4f}")
print(f"R2 Test: {test_r2:.4f}")


[I 2024-05-15 19:35:15,479] A new study created in memory with name: no-name-f3663428-107f-47e1-b913-a0d376da07d9
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_40157/1286862207.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_40157/1286862207.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
/opt/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stop

Mejores parámetros: {'lambda': 0.00026287363350301106, 'alpha': 0.00019296701697518064, 'colsample_bytree': 0.9985385048492151, 'subsample': 0.10140262844001847, 'learning_rate': 0.017395738891126174, 'n_estimators': 569, 'max_depth': 15, 'min_child_weight': 48}
R2 Train: 0.0090
R2 Test: -0.0136
